<div class="alert alert-danger">

**Read the `Instructions` notebook** before you start working on this problem set! It contains instructions on how to create the submission package. If you need more details on the *BayesNet* class, have a look at the **`BayesNet Introduction` notebook** of Problem Set 2.
    
</div>

**General Remarks**:
- Do not delete or add cells.
- Store your results into the corresponding result variables or implement the provided function stubs.
- Replace the placeholders `# YOUR CODE HERE` `raise NotImplementedError()` / `YOUR ANSWER HERE` with your code / answers.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from bayesian_network import BayesNet
from utils import sample_forward, get_default_bayes_net

# Parameter Learning

In this problem, we will assume that a fixed dependency graph structure between variables is given and learn the parameters (the complete Conditional Probability Distribution Table (CPDT)) from a set of events. Furthermore, we will use log-likelihood to find a model structure that also generalizes to future data.
    
## ML Estimates for Conditional Distributions

<div class="alert alert-warning">
    Implement the <i>maximum_likelihood_estimate</i> function, which computes the Maximum Likelihood Estimate for the parameters of a discrete (conditional) probability distribution $ P(X \mid \mathit{pa}(X) )$, given a data set. (3 points)
</div>

`maximum_likelihood_estimate` takes  three parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `variable_id` is the column index of the variable to estimate the distribution for.
- `parent_ids` is a tuple containing the column indices of parent variables.

`maximum_likelihood_estimate` must return one object:
- A Maximum Likelihood Estimate (MLE) of the parameters in the form of a `np.ndarray`. The first dimension (index `0`) of the returned array must correspond to variable `variable_id`, the remaining dimensions must be sorted according to `parent_ids`. Altogether, tuple `(variable_id, ) + parent_ids` gives the mapping of dimensions to variables.

**Hints**:
- **Assume that all variables are binary.**
- To count elements in a Numpy array, you simply loop over the data array.
- Do not forget to use `keepdims=True` when normalizing your CPDT.

In [2]:
def maximum_likelihood_estimate(data: np.ndarray, variable_id: int, parent_ids: tuple=tuple()):
    """
    Estimates the conditional probability distribution of a (discrete) variable from data.
    :param data:         data to estimate distribution from
    :param variable_id:  column index corresponding to the variable we estimate the distribution for
    :param parent_ids:   column indices of the variables the distribution is conditioned on
    :returns:            estimated conditional probability distribution table
    """
    
    assert type(variable_id) == int
    assert type(parent_ids) == tuple
    
    # mapping of axis to variable_id,
    # e.g. the variable with id variable_ids[i] is on axis i of the CPDT
    variable_ids = (variable_id,) + parent_ids
    
    # YOUR CODE HERE
    counts = np.zeros([2] * len(variable_ids))
    
    for x in data:
        if len(parent_ids) > 0:
            counts[x[variable_id], x[np.array(parent_ids)]] += 1
        else:
            counts[x[variable_id]] += 1
    
    cpdt = counts / counts.sum(0)
    
    return cpdt

In [3]:
# sanity checks
_A_, _B_, _C_, _D_, _E_ = 0, 1, 2, 3, 4
# get the bayes net from the previous problem
bayes_net = get_default_bayes_net()
np.random.seed(0)
# draw 100 samples
data = sample_forward(bayes_net, 100)

# get exact A from bayes net
expected = bayes_net[_A_].pdt[:,0,0,0,0]
# estimate A from the data
actual = maximum_likelihood_estimate(data, _A_)
# estimate should not be far off
assert expected.shape == actual.shape, f'Shape of PDT was {actual.shape} but expected {expected.shape}.'
assert np.allclose(expected, actual, atol=0.05), f'Expected {expected} but got {actual} instead.'

# get exact B_A from bayes net
expected = bayes_net[_B_].pdt[:,:,0,0,0].T
# estimate B_A from data
actual = maximum_likelihood_estimate(data, _B_, (_A_,))
# estimate should not be far off
assert expected.shape == actual.shape, f'Shape of CPDT was {actual.shape} but expected {expected.shape}.'
assert np.allclose(expected, actual, atol=0.05), f'Expected {expected} but got {actual} instead.'

### The Log-Likelihood Function

<div class="alert alert-warning">
    Implement the <i>log_likelihood</i> function, which computes the log-likelihood $\mathcal{L}(\mathcal{M} : \mathcal{D})$ of a model (BayesNet) realtive to a data set. (3 points)
</div>

`log_likelihood`takes  two parameters:
- `data` is a NumPy array of shape `(num_samples, num_variables)`.
- `bayes_net` a BayesNet object representing the model $\mathcal{M}$ (containing already estimated CPDTs).

`log_likelihood` must return one object:
- The log-likelihood of the model given the data (i.e., a floating-point number $\leq 0$).

Hint:
- Recall that iterating over the variables in the BayesNet is super easy: `for variable in bayes_net: ...`.
- The 1D probability distribution of variable $X$ given its parents $\mathit{pa}(X)$, $P(X \mid \mathit{pa}(X))$, can be obtained by passing the random event to the variable, i.e., `variable(data[i])`.
- Use the natural logarithm for your computations, i.e. `np.log`.

In [4]:
def log_likelihood(data: np.ndarray, bayes_net: BayesNet) -> float:
    """
    Computes the log-likelihood of a given Bayesian network relative to the data.
    :param data:      data to compute the log-likelihood relative to.
    :param bayes_net: Bayesian network model.
    :returns:         the log-likelihood of the Bayesian network relative to the data.
    """    

    ll = 0
    
    # YOUR CODE HERE
    for x in data:
        for variable in bayes_net:
            distribution = variable(x)
            ll += np.log(distribution[x[variable.id]])
    
    return ll

In [5]:
# sanity checks
# get the bayes net from the previous problem
bayes_net = get_default_bayes_net()
np.random.seed(0)
# draw 100 samples
data = sample_forward(bayes_net, 100)

# expected log-likelihood
expected = -215.9
# actual log-likelihood
actual = log_likelihood(data, bayes_net)

# must be close
assert isinstance(actual, float), f'Log Likelihood should be of type `float`, but got {type(actual)}.'
assert np.isclose(expected, actual, atol=0.1), f'Expected Log-Likelihood {expected}, but got {actual}.'


# remove unused variables
del data
del bayes_net

## Finding a Model for Strokes   

After watching hours of medical dramas on television, a medicine freshman tries to figure out the perfect prediction model for strokes. Some of her computer science colleagues told her about how Bayesian networks can be used for symptom diagnosis, so she decides to model her ideas using this technique. In order to find out the (conditional) probability distributions, she needs to find a lot of training examples. One of her computer science colleagues cracks the computer system of the university clinic and copies a lot of medical patient data. 

All variables in this example are boolean (false=0 or true=1). The data for this assignment is stored in two text files, "train.txt" and "test.txt". They contain 500 (train) and 5000 (test) samples of the following five random variables:

 - Column 0: A ... Alcoholism
 - Column 1: H ... High Blood Pressure
 - Column 2: S ... Stroke
 - Column 3: C ... Confusion
 - Column 4: V ... Vertigo
 
First, she decides to try out the following, very simple model:
    
<img style='width:100%;  max-width:400px;' src="img/bn_mod1.svg">

<br>

<div class="alert alert-warning">
    Estimate the (conditional) probability tables of Model 1 and compute the log-likelihood of Model 1 given the training data. (1 point)
</div>

Store the estimated CPDTs into the provided variables.

**Hints**:
- Use the two functions you implemented above (`maximum_likelihood_estimate` and `log_likelihood`)!
- The training data is stored in variable `train`.
- `_A_, _H_, _S_, _C_, _V_` hold the column indices (= IDs) of the variables. 

In [6]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4
train = np.loadtxt('data/train.txt', dtype=int)

A, H, S, C, V = None, None, None, None, None

# YOUR CODE HERE
import itertools

def conditional_distribution(data, variable_id, parent_ids=[]):
    distribution = np.zeros([2] * (len(parent_ids) + 1))
    
    for permutation in itertools.product([0, 1], repeat=len(parent_ids) + 1):
        if len(parent_ids) == 0:
            matching_values = data
        else:
            matching_values = data[np.logical_and.reduce([data[:, parent_id] == value for parent_id, value in zip(parent_ids, permutation[1:])])]
        
        distribution[permutation] = np.count_nonzero(matching_values[:, variable_id] == permutation[0]) / len(matching_values)
    
    return distribution

A = conditional_distribution(train, _A_)
H = conditional_distribution(train, _H_)
S = conditional_distribution(train, _S_)
C = conditional_distribution(train, _C_)
V = conditional_distribution(train, _V_)

# begin sanity check
assert np.allclose(A.sum(axis=0), 1), '`A` must be a well-formed PDT (sum to 1)'
assert np.allclose(H.sum(axis=0), 1), '`H` must be a well-formed PDT (sum to 1)'
assert np.allclose(S.sum(axis=0), 1), '`S` must be a well-formed PDT (sum to 1)'
assert np.allclose(C.sum(axis=0), 1), '`C` must be a well-formed PDT (sum to 1)'
assert np.allclose(V.sum(axis=0), 1), '`V` must be a well-formed PDT (sum to 1)'
# end sanity check

bayes_net_1 = BayesNet(
    (A, (_A_,)),
    (H, (_H_,)),
    (S, (_S_,)),
    (C, (_C_,)),
    (V, (_V_,))
)

tr_log_likelihood_1 = log_likelihood(train, bayes_net_1)

In [7]:
# sanity check
assert -1300 < tr_log_likelihood_1 < -1100, f'Expected log-likelihood to be in [-1300, -1100] but got {tr_log_likelihood_1}.'


Not satisfied with the results, our freshman decides that Model 1 is probably too simple to represent the real world. She adds some edges to the model and tries again:

<img  style='width:100%;  max-width:400px;' src="img/bn_mod2.svg">

<div class="alert alert-warning">
    Estimate the (conditional) probability tables of Model 2 and compute the log-likelihood of Model 2 given the training data. (1 point)
</div>

Store the estimated CPDTs into the provided variables. The dimensions of the CPDT must be sorted according to the naming of the variable, e.g., in C_AS, dimension 0 corresponds to C, dimension 1 to A, and dimension 2 to S.

**Hints**:
- Use the two functions you implemented above (`maximum_likelihood_estimate` and `log_likelihood`)!
- The training data is stored in variable `train`. 
- `_A_, _H_, _S_, _C_, _V_` hold the column indices (= IDs) of the variables. 

In [8]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4
train = np.loadtxt('data/train.txt', dtype=int)

A, H_A, S_H, C_AS, V_S = None, None, None, None, None

# YOUR CODE HERE
A = conditional_distribution(train, _A_)
H_A = conditional_distribution(train, _H_, [_A_])
S_H = conditional_distribution(train, _S_, [_H_])
C_AS = conditional_distribution(train, _C_, [_A_, _S_])
V_S = conditional_distribution(train, _V_, [_S_])

# begin sanity check
assert np.allclose(A.sum(axis=0), 1), '`A` must be a well-formed PDT (sum to 1)'
assert np.allclose(H_A.sum(axis=0), 1), '`H_A` must be a well-formed CPDT (sum to 1)'
assert np.allclose(S_H.sum(axis=0), 1), '`S_H` must be a well-formed CPDT (sum to 1)'
assert np.allclose(C_AS.sum(axis=0), 1), '`C_AS` must be a well-formed CPDT (sum to 1)'
assert np.allclose(V_S.sum(axis=0), 1), '`V_S` must be a well-formed CPDT (sum to 1)'
# end sanity check

bayes_net_2 = BayesNet(
    (A, (_A_,)),
    (H_A, (_H_,_A_)),
    (S_H, (_S_,_H_)),
    (C_AS, (_C_,_A_,_S_)),
    (V_S, (_V_,_S_))
)

tr_log_likelihood_2 = log_likelihood(train, bayes_net_2)

In [9]:
# sanity check
assert -1300 < tr_log_likelihood_2 < -1100, f'Expected log-likelihood to be in [-1300, -1100] but got {tr_log_likelihood_2}.'



Finally, she decides to try out an even more complex model:

<img  style='width:100%;  max-width:400px;' src="img/bn_mod3.svg">

<div class="alert alert-warning">
    Estimate the (conditional) probability tables of Model 3 and compute the log-likelihood of Model 3 given the training data. (1 point)
</div>

Store the CPDTs into the provided variables. The dimensions of the CPDT must be sorted according to the naming of the variable, e.g., in C_AS, dimension 0 corresponds to C, dimension 1 to A, and dimension 2 to S.

**Hint**:
- Use the two functions you implemented above (`maximum_likelihood_estimate` and `log_likelihood`)!
- The training data is stored in variable `train`. 
- `_A_, _H_, _S_, _C_, _V_` hold the column indices (= IDs) of the variables. 

In [10]:
_A_, _H_, _S_, _C_, _V_ = 0, 1, 2, 3, 4
train = np.loadtxt('data/train.txt', dtype=int)

A, H_A, S_AH, C_AS, V_CS = None, None, None, None, None

# YOUR CODE HERE
A = conditional_distribution(train, _A_)
H_A = conditional_distribution(train, _H_, [_A_])
S_AH = conditional_distribution(train, _S_, [_A_, _H_])
C_AS = conditional_distribution(train, _C_, [_A_, _S_])
V_CS = conditional_distribution(train, _V_, [_C_, _S_])

# begin sanity check
assert np.allclose(A.sum(axis=0), 1), '`A` must be a well-formed PDT (sum to 1)'
assert np.allclose(H_A.sum(axis=0), 1), '`H_A` must be a well-formed CPDT (sum to 1)'
assert np.allclose(S_AH.sum(axis=0), 1), '`S_AH` must be a well-formed CPDT (sum to 1)'
assert np.allclose(C_AS.sum(axis=0), 1), '`C_AS` must be a well-formed CPDT (sum to 1)'
assert np.allclose(V_CS.sum(axis=0), 1), '`V_CS` must be a well-formed CPDT (sum to 1)'
# end sanity check

bayes_net_3 = BayesNet(
    (A, (_A_,)),
    (H_A, (_H_,_A_)),
    (S_AH, (_S_,_A_,_H_)),
    (C_AS, (_C_,_A_,_S_)),
    (V_CS, (_V_,_C_,_S_))
)

tr_log_likelihood_3 = log_likelihood(train, bayes_net_3)

In [11]:
# sanity check
assert -1300 < tr_log_likelihood_3 < -1100, f'Expected log-likelihood to be in [-1300, -1100] but got {tr_log_likelihood_3}.'


### Compare Train Log-Likelihoods

Compare the log-likelihoods of the training data given the three models.

In [12]:
print('logP(train|M1) = {}'.format(tr_log_likelihood_1))
print('logP(train|M2) = {}'.format(tr_log_likelihood_2))
print('logP(train|M3) = {}'.format(tr_log_likelihood_3))

logP(train|M1) = -1214.6425612469914
logP(train|M2) = -1175.123630631206
logP(train|M3) = -1174.6821826610374


### Compare Test Log-Likelihoods

The computer science colleague manages to obtain more data from the clinic's network and stores it in the file `test.txt`. Our medicine freshman is eager to try the models on the new data.

In [13]:
test = np.loadtxt('data/test.txt', dtype=int)

She computes the log-likelihood of the test data for each of the three models:

In [14]:
te_log_likelihood_1 = log_likelihood(test, bayes_net_1)
te_log_likelihood_2 = log_likelihood(test, bayes_net_2)
te_log_likelihood_3 = log_likelihood(test, bayes_net_3)

print('logP(test|M1) = {}'.format(te_log_likelihood_1))
print('logP(test|M2) = {}'.format(te_log_likelihood_2))
print('logP(test|M3) = {}'.format(te_log_likelihood_3))

logP(test|M1) = -11911.976345359737
logP(test|M2) = -11565.57072258751
logP(test|M3) = -11571.024593414042


<div class="alert alert-warning">
    Answer the following questions by storing True or False into the provided variables! (0.5 points each)
</div>

1. In maximum likelihood estimation, variables with a large number of parents might lead to unspecified distributions. 
2. A higher (log-)likelihood on the training data always translates to better performance on unseen future data.
3. (Log-)likelihood describes the probability of the model as a function of the data.
4. Adding an edge to a Bayesian network will never result in a lower likelihood on the training data.
5. Out of the three tested models, model 3 is the best choice.
6. If we use a uniform prior over model parameters, the Maximum A Posteriori (MAP) estimate is equal to the Maximum Likelihood (ML) estimate, i.e., $\hat{\theta}_{MAP} = \hat{\theta}_{ML}$.


In [15]:
results = dict()
results['1'] = False
results['2'] = True
results['3'] = False
results['4'] = True
results['5'] = True
results['6'] = False


In [16]:
# this is a hidden test; don't remove ...

In [17]:
# this is a hidden test; don't remove ...

In [18]:
# this is a hidden test; don't remove ...

In [19]:
# this is a hidden test; don't remove ...

In [20]:
# this is a hidden test; don't remove ...

In [21]:
# this is a hidden test; don't remove ...